In [38]:
from pyspark.sql import functions as F


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IngestaNYCTaxiRaw") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .getOrCreate()

print(" Spark iniciado correctamente")


 Spark iniciado correctamente


In [4]:
import os

pg_url = f"jdbc:postgresql://{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
pg_props = {
    "user": os.getenv("PG_USER"),
    "password": os.getenv("PG_PASSWORD"),
    "driver": "org.postgresql.Driver"
}

schema_raw = os.getenv("PG_SCHEMA_RAW")
print(f"Conectando a Postgres → esquema: {schema_raw}")


Conectando a Postgres → esquema: raw


In [6]:
from pyspark.sql import functions as F
from datetime import datetime
import os

def load_parquet_to_postgres_with_log(parquet_path, table_name):
    """
    Carga un archivo Parquet a una tabla Postgres dentro del esquema RAW.
    Registra conteo de filas y timestamp UTC de ingesta.
    """
    try:
        schema_raw = os.getenv("PG_SCHEMA_RAW")
        print(f"Iniciando carga de: {parquet_path} → {schema_raw}.{table_name}")
        start_time = datetime.utcnow()

        # Leer archivo Parquet
        df = spark.read.parquet(parquet_path)

        # Agregar columna de metadatos
        df = df.withColumn("ingested_at_utc", F.lit(start_time.strftime("%Y-%m-%d %H:%M:%S")))

        # Escribir en Postgres
        df.write.jdbc(
            url=pg_url,
            table=f"{schema_raw}.{table_name}",
            mode="append",
            properties=pg_props
        )

        # Log de filas
        row_count = df.count()
        print(f"{table_name}: {row_count} filas insertadas. ({start_time})")

    except Exception as e:
        print(f"Error cargando {table_name}: {e}")


In [6]:
import os
import requests

# Carpeta dentro del contenedor (vinculada a ./data del host)
data_dir = "/data"
os.makedirs(data_dir, exist_ok=True)

# URL base del dataset oficial del NYC TLC (formato Parquet)
#  Todos los enlaces de la página oficial redirigen a este mismo dominio.
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"

# Rango pequeño de prueba (solo 2 meses, un año)
# Para el deber completo: usar years = range(2015, 2026) y months = range(1, 13)
years = [2019]
months = [1, 2]
services = ["yellow", "green"]

def download_file(url, local_path):
    """Descarga el archivo desde la URL si no existe localmente."""
    if not os.path.exists(local_path):
        print(f"⬇ Descargando {os.path.basename(local_path)} ...")
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(local_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f" Guardado en {local_path}")
        else:
            print(f"Error {r.status_code}: {url}")
    else:
        print(f"Ya existe: {os.path.basename(local_path)}")

# Descarga iterativa de ambos servicios y meses definidos
for service in services:
    for year in years:
        for month in months:
            file_name = f"{service}_tripdata_{year}-{month:02d}.parquet"
            url = base_url + file_name
            local_path = os.path.join(data_dir, file_name)
            download_file(url, local_path)

#Archivo de zonas (solo uno, requerido para la OBT)
zones_url = base_url + "taxi+_zone_lookup.parquet"
zones_path = os.path.join(data_dir, "taxi+_zone_lookup.parquet")
download_file(zones_url, zones_path)

print("\nDescarga finalizada (2 meses de prueba).")
print("Para todo el rango 2015–2025, cambia las líneas:")
print("years = range(2015, 2026)")
print("months = range(1, 13)")



⬇ Descargando yellow_tripdata_2019-01.parquet ...
 Guardado en /data/yellow_tripdata_2019-01.parquet
⬇ Descargando yellow_tripdata_2019-02.parquet ...
 Guardado en /data/yellow_tripdata_2019-02.parquet
⬇ Descargando green_tripdata_2019-01.parquet ...
 Guardado en /data/green_tripdata_2019-01.parquet
⬇ Descargando green_tripdata_2019-02.parquet ...
 Guardado en /data/green_tripdata_2019-02.parquet
⬇ Descargando taxi+_zone_lookup.parquet ...
Error 403: https://d37ci6vzurychx.cloudfront.net/trip-data/taxi+_zone_lookup.parquet

Descarga finalizada (2 meses de prueba).
Para todo el rango 2015–2025, cambia las líneas:
years = range(2015, 2026)
months = range(1, 13)


In [16]:
spark.read.parquet("/data/yellow_tripdata_2019-01.parquet").show(5)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.6

In [2]:
import os
import requests
import pandas as pd
from pyspark.sql import SparkSession

# Ruta y URL oficial actualizada
data_dir = "/data"
os.makedirs(data_dir, exist_ok=True)

zones_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
csv_path = os.path.join(data_dir, "taxi_zone_lookup.csv")
parquet_path = os.path.join(data_dir, "taxi_zone_lookup.parquet")

# Descargar CSV
print("Descargando taxi_zone_lookup.csv ...")
r = requests.get(zones_url, stream=True)
if r.status_code == 200:
    with open(csv_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"CSV guardado en {csv_path}")
else:
    print(f"Error {r.status_code} descargando el archivo.")

# Convertir CSV a Parquet
if os.path.exists(csv_path):
    df_pd = pd.read_csv(csv_path)
    spark_df = spark.createDataFrame(df_pd)
    spark_df.write.mode("overwrite").parquet(parquet_path)
    print(f"Archivo convertido a Parquet en {parquet_path}")
else:
    print("No se encontró el CSV para convertirlo.")


Descargando taxi_zone_lookup.csv ...
CSV guardado en /data/taxi_zone_lookup.csv
Archivo convertido a Parquet en /data/taxi_zone_lookup.parquet


In [17]:
import psycopg2
from psycopg2 import sql
import os

# Crear conexión
conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD")
)

conn.autocommit = True
cur = conn.cursor()

# Crear esquemas raw y analytics si no existen
cur.execute("CREATE SCHEMA IF NOT EXISTS raw;")
cur.execute("CREATE SCHEMA IF NOT EXISTS analytics;")

print("Esquemas 'raw' y 'analytics' creados/verificados correctamente.")

cur.close()
conn.close()


Esquemas 'raw' y 'analytics' creados/verificados correctamente.


In [3]:
import psycopg2
import os

# Conexión
conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD")
)
conn.autocommit = True
cur = conn.cursor()

# Eliminar tablas si existen (o vaciarlas)
cur.execute("DROP TABLE IF EXISTS raw.yellow_taxi_trip CASCADE;")
cur.execute("DROP TABLE IF EXISTS raw.green_taxi_trip CASCADE;")
cur.execute("DROP TABLE IF EXISTS raw.taxi_zone_lookup CASCADE;")

print("Tablas eliminadas correctamente (yellow, green, lookup).")

cur.close()
conn.close()


Tablas eliminadas correctamente (yellow, green, lookup).


In [10]:
import os

# Puedes ajustar estos rangos fácilmente 
years = [2019]          # o range(2015, 2026)
months = range(1, 3)    # meses del 1 al 2 (enero y febrero)
services = ["yellow", "green"]  # datasets a incluir

data_dir = "/data"
schema_raw = os.getenv("PG_SCHEMA_RAW")

def bulk_load_parquets_to_postgres():
    print(f"\nIniciando carga masiva hacia esquema '{schema_raw}'...\n")

    # Cargar taxis (Yellow y Green)
    for service in services:
        for year in years:
            for month in months:
                file_name = f"{service}_tripdata_{year}-{month:02d}.parquet"
                parquet_path = os.path.join(data_dir, file_name)

                if os.path.exists(parquet_path):
                    print(f" Encontrado: {file_name}")
                    load_parquet_to_postgres_with_log(parquet_path, f"{service}_taxi_trip")
                else:
                    print(f"No encontrado: {file_name}")

    # Cargar tabla de zonas (si existe)
    lookup_path = os.path.join(data_dir, "taxi_zone_lookup.parquet")
    if os.path.exists(lookup_path):
        load_parquet_to_postgres_with_log(lookup_path, "taxi_zone_lookup")
    else:
        print("taxi_zone_lookup.parquet no encontrado en /data")

    print("\nCarga masiva finalizada.\n")

bulk_load_parquets_to_postgres()



Iniciando carga masiva hacia esquema 'raw'...

 Encontrado: yellow_tripdata_2019-01.parquet
Iniciando carga de: /data/yellow_tripdata_2019-01.parquet → raw.yellow_taxi_trip
yellow_taxi_trip: 7696617 filas insertadas. (2025-11-11 18:43:23.411340)
 Encontrado: yellow_tripdata_2019-02.parquet
Iniciando carga de: /data/yellow_tripdata_2019-02.parquet → raw.yellow_taxi_trip
yellow_taxi_trip: 7049370 filas insertadas. (2025-11-11 18:44:53.487189)
 Encontrado: green_tripdata_2019-01.parquet
Iniciando carga de: /data/green_tripdata_2019-01.parquet → raw.green_taxi_trip
green_taxi_trip: 672105 filas insertadas. (2025-11-11 18:46:19.508169)
 Encontrado: green_tripdata_2019-02.parquet
Iniciando carga de: /data/green_tripdata_2019-02.parquet → raw.green_taxi_trip
green_taxi_trip: 615594 filas insertadas. (2025-11-11 18:46:27.449328)
Iniciando carga de: /data/taxi_zone_lookup.parquet → raw.taxi_zone_lookup
taxi_zone_lookup: 265 filas insertadas. (2025-11-11 18:46:34.879054)

Carga masiva finalizad

In [8]:
from pyspark.sql import SparkSession

def verify_postgres_table_light(table_name):
    """Verifica que la tabla existe en Postgres, mostrando conteo SQL y opcionalmente filas de muestra."""
    print(f"\nVerificando tabla: {table_name}")

    # Consultar el conteo directamente desde Postgres
    import psycopg2
    conn = psycopg2.connect(
        host=os.getenv("PG_HOST"),
        port=os.getenv("PG_PORT"),
        database=os.getenv("PG_DB"),
        user=os.getenv("PG_USER"),
        password=os.getenv("PG_PASSWORD")
    )
    cur = conn.cursor()
    cur.execute(f"SELECT COUNT(*) FROM {schema_raw}.{table_name};")
    count = cur.fetchone()[0]
    cur.close()
    conn.close()

    print(f"{table_name} contiene {count:,} filas.")

    # Mostrar 5 filas con Spark solo si hay memoria disponible
    try:
        df = (
            spark.read.jdbc(
                url=pg_url,
                table=f"{schema_raw}.{table_name}",
                properties=pg_props
            )
            .limit(5)
        )
        print(f"Muestra de datos ({table_name}):")
        df.show(5, truncate=False)
    except Exception as e:
        print(f"No se pudo mostrar muestra con Spark ({e}) — el conteo SQL sigue siendo válido.")

verify_postgres_table_light("yellow_taxi_trip")
verify_postgres_table_light("green_taxi_trip")
verify_postgres_table_light("taxi_zone_lookup")




Verificando tabla: yellow_taxi_trip
yellow_taxi_trip contiene 14,745,987 filas.
No se pudo mostrar muestra con Spark ([Errno 111] Connection refused) — el conteo SQL sigue siendo válido.

Verificando tabla: green_taxi_trip
green_taxi_trip contiene 1,287,699 filas.
No se pudo mostrar muestra con Spark ([Errno 111] Connection refused) — el conteo SQL sigue siendo válido.

Verificando tabla: taxi_zone_lookup
taxi_zone_lookup contiene 265 filas.
No se pudo mostrar muestra con Spark ([Errno 111] Connection refused) — el conteo SQL sigue siendo válido.


In [9]:
import psycopg2, os

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD")
)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS raw.ingestion_log (
    id SERIAL PRIMARY KEY,
    dataset_name VARCHAR(50),
    year INTEGER,
    month INTEGER,
    row_count BIGINT,
    ingested_at_utc TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")
conn.commit()
cur.close()
conn.close()

print("Tabla raw.ingestion_log creada correctamente.")


Tabla raw.ingestion_log creada correctamente.


In [10]:
import psycopg2, os
from datetime import datetime

# Registros de auditoría para las cargas ya realizadas
ingestas = [
    ("yellow_taxi_trip", 2019, 1, 7696617),
    ("yellow_taxi_trip", 2019, 2, 7049370),
    ("green_taxi_trip", 2019, 1, 672105),
    ("green_taxi_trip", 2019, 2, 615594),
    ("taxi_zone_lookup", 2019, None, 265)
]

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD")
)
cur = conn.cursor()

for dataset_name, year, month, row_count in ingestas:
    cur.execute("""
        INSERT INTO raw.ingestion_log (dataset_name, year, month, row_count, ingested_at_utc)
        VALUES (%s, %s, %s, %s, CURRENT_TIMESTAMP);
    """, (dataset_name, year, month, row_count))

conn.commit()
cur.close()
conn.close()

print("Auditoría registrada correctamente en raw.ingestion_log.")


🧾 Auditoría registrada correctamente en raw.ingestion_log.


In [15]:
from sqlalchemy import create_engine
import pandas as pd, os

# Crear motor SQLAlchemy con las variables del entorno
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
)

# Leer la tabla de auditoría
df_log = pd.read_sql("SELECT * FROM raw.ingestion_log ORDER BY id;", engine)
df_log



,id,dataset_name,year,month,row_count,ingested_at_utc
0,1,yellow_taxi_trip,2019,1.0,7696617,2025-11-11 21:42:12.298545
1,2,yellow_taxi_trip,2019,2.0,7049370,2025-11-11 21:42:12.298545
2,3,green_taxi_trip,2019,1.0,672105,2025-11-11 21:42:12.298545
3,4,green_taxi_trip,2019,2.0,615594,2025-11-11 21:42:12.298545
4,5,taxi_zone_lookup,2019,NaN,265,2025-11-11 21:42:12.298545
